In [27]:
#####################################
### Carregar arquivos no supabase ###
#####################################

In [28]:
# imports
from supabase import create_client, Client
import pandas as pd
import os

In [29]:
# Conexão com Supabase
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)


In [30]:
# Carregar dados tratados
df_to_load = pd.read_csv("../data/processed/base_vendas_tratada.csv")

In [31]:
res = supabase.table("vendas").select("*").execute()
df_supabase_atual = pd.DataFrame(res.data)

In [32]:
chave = ["Data da Venda", "Produto", "Cliente", "Total (R$)"]

In [33]:
df_merge = df_to_load.merge(df_supabase_atual[chave], on=chave, how="left", indicator=True)
df_para_inserir = df_merge[df_merge["_merge"] == "left_only"].drop(columns=["_merge"])


In [34]:
print(f"📥 Linhas novas detectadas: {len(df_para_inserir)}")


📥 Linhas novas detectadas: 0


In [35]:
# Inserir no Supabase (Em lote)

if not df_para_inserir.empty:
    df_para_inserir = df_para_inserir.fillna(None)
    dados = df_para_inserir.to_dict(orient="records")
    supabase.table("vendas").insert(dados).execute()
    print("✅ Novos dados inseridos com sucesso!")
else:
    print("✅ Nenhuma nova linha para inserir.")

✅ Nenhuma nova linha para inserir.
